In [ ]:
# Install required packages
!pip install pandas numpy matplotlib seaborn scikit-learn groq python-dotenv tqdm transformers torch -q 

In [ ]:
# Import libraries
import warnings
import os

# Suppress FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import time
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    classification_report,
    matthews_corrcoef,
)

# API setup
from groq import Groq
from dotenv import load_dotenv
from transformers import pipeline
import torch

# Load environment variables
load_dotenv()

# Configure APIs
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if GROQ_API_KEY:
    groq_client = Groq(api_key=GROQ_API_KEY)

# Load FinBERT model
print("Loading FinBERT model...")
device = 0 if torch.cuda.is_available() else -1
finbert_pipeline = pipeline(
    "sentiment-analysis", model="ProsusAI/finbert", device=device
)

# Set plot style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (14, 6)

print("✓ Libraries imported successfully")
print(f"✓ Groq API configured: {bool(GROQ_API_KEY)}")
print(f"✓ FinBERT loaded on {'GPU' if device == 0 else 'CPU'}")

## 1. Load Dataset

In [ ]:
# Load the 100% agreement dataset (highest quality)
data_path = "../../DatasetAnalysis_FinancialPhraseBank/FinancialPhraseBank-v1.0/Sentences_AllAgree.txt"

sentences = []
sentiments = []

with open(data_path, "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        line = line.strip()
        if "@" in line:
            parts = line.rsplit("@", 1)
            if len(parts) == 2:
                sentences.append(parts[0])
                sentiments.append(parts[1])

df = pd.DataFrame({"sentence": sentences, "true_sentiment": sentiments})

print(f"Dataset loaded: {len(df)} sentences")
print(f"\nSentiment distribution:")
print(df["true_sentiment"].value_counts())

# Display sample
print("\n" + "=" * 80)
print("Sample sentences:")
print("=" * 80)
display(df.sample(5, random_state=42))

## 2. Zero-Shot Prompt Design

**Prompt Strategy**: Simple, direct instruction with no examples. Enforces strict JSON output format.

In [ ]:
def create_zero_shot_prompt(sentence):
    """
    Creates a zero-shot prompt for sentiment classification.
    No examples provided - model relies on pretrained knowledge.
    """
    prompt = f"""You are a financial sentiment analysis expert. Analyze the sentiment of financial statements with precision.

Task: Classify the sentiment of the following financial statement as "positive", "negative", or "neutral" from an investor's perspective.

Guidelines:
- Positive: Financial improvements, growth, profits, revenue increases, cost reductions, successful expansions
- Negative: Financial declines, losses, revenue drops, cost increases, failed ventures, layoffs, legal issues
- Neutral: Factual statements with no clear financial impact, routine announcements, or balanced mixed signals

Financial Statement:
"{sentence}"

Return ONLY valid JSON in this exact format:
{{
    "sentiment": "positive/negative/neutral",
    "confidence": 0.0-1.0,
    "rationale": "Brief explanation in one sentence"
}}
"""
    return prompt


# Test prompt
test_sentence = "Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007."
print("=" * 80)
print("ZERO-SHOT PROMPT EXAMPLE")
print("=" * 80)
print(create_zero_shot_prompt(test_sentence))

## 3. Model Inference Functions

In [ ]:
def call_llama(prompt, model_name, temperature=0.0):
    """Call Llama via Groq API"""
    max_retries = 3
    last_error = None
    for attempt in range(max_retries):
        try:
            chat_completion = groq_client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ],
                model=model_name,
                temperature=temperature,
                max_tokens=500,
            )
            return chat_completion.choices[0].message.content
        except Exception as e:
            last_error = str(e)
            print(f"Attempt {attempt + 1}/{max_retries} failed: {last_error}")
            if attempt < max_retries - 1:
                time.sleep(2**attempt)
                continue
            return None
    return None


def parse_response(response_text):
    """Parse JSON response from model"""
    try:
        # Try to extract JSON from response
        if "```json" in response_text:
            json_str = response_text.split("```json")[1].split("```")[0].strip()
        elif "```" in response_text:
            json_str = response_text.split("```")[1].strip()
        else:
            json_str = response_text.strip()
        result = json.loads(json_str)
        return result
    except Exception as e:
        print("Parse error: {}".format(str(e)[:100]))
        print("Raw response was:\n{}".format(response_text))
        # Fallback: try to extract sentiment with regex
        response_lower = response_text.lower() if response_text else ""
        if "positive" in response_lower and "negative" not in response_lower:
            return {
                "sentiment": "positive",
                "confidence": 0.5,
                "rationale": "Parsed from text",
            }
        elif "negative" in response_lower:
            return {
                "sentiment": "negative",
                "confidence": 0.5,
                "rationale": "Parsed from text",
            }
        elif "neutral" in response_lower:
            return {
                "sentiment": "neutral",
                "confidence": 0.5,
                "rationale": "Parsed from text",
            }
        return None


def call_finbert(sentence):
    """Call FinBERT for sentiment classification"""
    try:
        result = finbert_pipeline(sentence[:512])  # FinBERT max length
        label_map = {
            "positive": "positive",
            "negative": "negative",
            "neutral": "neutral",
        }
        return {
            "sentiment": label_map.get(result[0]["label"].lower(), "neutral"),
            "confidence": result[0]["score"],
            "rationale": f"FinBERT classification: {result[0]['label']}",
        }
    except Exception as e:
        print(f"FinBERT error: {str(e)[:100]}")
        return None


print("✓ Model inference functions defined")

In [ ]:
# Test API calls with a sample sentence
test_sentence = "Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007."
test_prompt = create_zero_shot_prompt(test_sentence)

print("=" * 80)
print("Testing Mixtral-8x7B...")
print("=" * 80)
mixtral_response = call_llama(test_prompt, model_name="mixtral-8x7b-32768")
if mixtral_response:
    print(f"✓ Response received (length: {len(mixtral_response)})")
    print(f"Response: {mixtral_response[:500]}")
    parsed = parse_response(mixtral_response)
    if parsed:
        print(f"✓ Parsed successfully: {parsed}")
    else:
        print("✗ Failed to parse response")
else:
    print("✗ Mixtral call failed - no response received")

print("\n" + "=" * 80)
print("Testing Llama-3.1-70B...")
print("=" * 80)
llama_response = call_llama(test_prompt, model_name="llama-3.1-70b-versatile")
if llama_response:
    print(f"✓ Response received (length: {len(llama_response)})")
    print(f"Response: {llama_response[:500]}")
    parsed = parse_response(llama_response)
    if parsed:
        print(f"✓ Parsed successfully: {parsed}")
    else:
        print("✗ Failed to parse response")
else:
    print("✗ Llama call failed - no response received")

print("\n" + "=" * 80)
print("Testing FinBERT...")
print("=" * 80)
finbert_result = call_finbert(test_sentence)
if finbert_result:
    print(f"✓ FinBERT result: {finbert_result}")
else:
    print("✗ FinBERT call failed")

## 4. Run Experiments

### E1: Mixtral-8x7B (Zero-Shot)

In [ ]:
# Run on full dataset
test_df = df.copy()

# E1: Mixtral-8x7B
print("Running E1: Mixtral-8x7B (Zero-Shot)...")
e1_results = []

for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="E1 Progress"):
    prompt = create_zero_shot_prompt(row["sentence"])
    response = call_llama(prompt, model_name="mixtral-8x7b-32768")

    if response:
        parsed = parse_response(response)
        if parsed:
            e1_results.append(
                {
                    "sentence": row["sentence"],
                    "true_sentiment": row["true_sentiment"],
                    "predicted_sentiment": parsed.get("sentiment", "unknown"),
                    "confidence": parsed.get("confidence", 0),
                    "rationale": parsed.get("rationale", ""),
                }
            )
        else:
            e1_results.append(
                {
                    "sentence": row["sentence"],
                    "true_sentiment": row["true_sentiment"],
                    "predicted_sentiment": "error",
                    "confidence": 0,
                    "rationale": "Parse error",
                }
            )

    time.sleep(0.5)  # Rate limiting

e1_df = pd.DataFrame(e1_results)
print(f"\n✓ E1 completed: {len(e1_df)} predictions")
display(e1_df.head())

### E2: Llama-3.1-70B (Zero-Shot)

In [ ]:
# E2: Llama-3.1-70B (Zero-Shot)
print("Running E2: Llama-3.1-70B (Zero-Shot)...")
e2_results = []

for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="E2 Progress"):
    prompt = create_zero_shot_prompt(row["sentence"])
    response = call_llama(prompt, model_name="llama-3.1-70b-versatile")

    if response:
        parsed = parse_response(response)
        if parsed:
            e2_results.append(
                {
                    "sentence": row["sentence"],
                    "true_sentiment": row["true_sentiment"],
                    "predicted_sentiment": parsed.get("sentiment", "unknown"),
                    "confidence": parsed.get("confidence", 0),
                    "rationale": parsed.get("rationale", ""),
                }
            )
        else:
            e2_results.append(
                {
                    "sentence": row["sentence"],
                    "true_sentiment": row["true_sentiment"],
                    "predicted_sentiment": "error",
                    "confidence": 0,
                    "rationale": "Parse error",
                }
            )

    time.sleep(0.5)

e2_df = pd.DataFrame(e2_results)
print(f"\n✓ E2 completed: {len(e2_df)} predictions")
display(e2_df.head())

### E3: FinBERT (Zero-Shot)

In [ ]:
# E3: FinBERT (Zero-Shot)
print("Running E3: FinBERT (Zero-Shot)...")
e3_results = []

for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="E3 Progress"):
    result = call_finbert(row["sentence"])

    if result:
        e3_results.append(
            {
                "sentence": row["sentence"],
                "true_sentiment": row["true_sentiment"],
                "predicted_sentiment": result.get("sentiment", "unknown"),
                "confidence": result.get("confidence", 0),
                "rationale": result.get("rationale", ""),
            }
        )
    else:
        e3_results.append(
            {
                "sentence": row["sentence"],
                "true_sentiment": row["true_sentiment"],
                "predicted_sentiment": "error",
                "confidence": 0,
                "rationale": "FinBERT error",
            }
        )

e3_df = pd.DataFrame(e3_results)
print(f"\n✓ E3 completed: {len(e3_df)} predictions")
display(e3_df.head())

## 5. Calculate Metrics

In [ ]:
def calculate_metrics(df, exp_name):
    """Calculate all evaluation metrics"""
    # Check if dataframe is empty or missing required columns
    if df.empty or "predicted_sentiment" not in df.columns:
        print(f"⚠️ Warning: {exp_name} has no valid predictions!")
        return (
            {
                "Experiment": exp_name,
                "Total Samples": 0,
                "Valid Predictions": 0,
                "Accuracy": 0,
                "Macro-F1": 0,
                "Weighted-F1": 0,
                "Macro-Precision": 0,
                "Macro-Recall": 0,
                "MCC": 0,
                "Positive_Precision": 0,
                "Positive_Recall": 0,
                "Positive_F1": 0,
                "Negative_Precision": 0,
                "Negative_Recall": 0,
                "Negative_F1": 0,
                "Neutral_Precision": 0,
                "Neutral_Recall": 0,
                "Neutral_F1": 0,
            },
            np.zeros((3, 3)),
            pd.DataFrame(),
        )

    # Filter out errors
    valid_df = df[
        df["predicted_sentiment"].isin(["positive", "negative", "neutral"])
    ].copy()

    # Check if we have valid predictions
    if valid_df.empty:
        print(f"⚠️ Warning: {exp_name} has no valid predictions after filtering!")
        return (
            {
                "Experiment": exp_name,
                "Total Samples": len(df),
                "Valid Predictions": 0,
                "Accuracy": 0,
                "Macro-F1": 0,
                "Weighted-F1": 0,
                "Macro-Precision": 0,
                "MCC": 0,
                "Positive_Precision": 0,
                "Positive_Recall": 0,
                "Positive_F1": 0,
                "Negative_Precision": 0,
                "Negative_Recall": 0,
                "Negative_F1": 0,
                "Neutral_Precision": 0,
                "Neutral_Recall": 0,
                "Neutral_F1": 0,
            },
            np.zeros((3, 3)),
            pd.DataFrame(),
        )

    y_true = valid_df["true_sentiment"]
    y_pred = valid_df["predicted_sentiment"]

    metrics = {
        "Experiment": exp_name,
        "Total Samples": len(df),
        "Valid Predictions": len(valid_df),
        "Accuracy": accuracy_score(y_true, y_pred),
        "Macro-F1": f1_score(y_true, y_pred, average="macro"),
        "Weighted-F1": f1_score(y_true, y_pred, average="weighted"),
        "Macro-Precision": precision_score(y_true, y_pred, average="macro"),
        "Macro-Recall": recall_score(y_true, y_pred, average="macro"),
        "MCC": matthews_corrcoef(y_true, y_pred),
    }

    # Per-class metrics
    labels = ["positive", "negative", "neutral"]
    precision_per_class = precision_score(
        y_true, y_pred, labels=labels, average=None, zero_division=0
    )
    recall_per_class = recall_score(
        y_true, y_pred, labels=labels, average=None, zero_division=0
    )
    f1_per_class = f1_score(
        y_true, y_pred, labels=labels, average=None, zero_division=0
    )

    for i, label in enumerate(labels):
        metrics[f"{label.capitalize()}_Precision"] = precision_per_class[i]
        metrics[f"{label.capitalize()}_Recall"] = recall_per_class[i]
        metrics[f"{label.capitalize()}_F1"] = f1_per_class[i]

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    return metrics, cm, valid_df


# Calculate metrics for all experiments
e1_metrics, e1_cm, e1_valid = calculate_metrics(e1_df, "E1: Mixtral-8x7B")
e2_metrics, e2_cm, e2_valid = calculate_metrics(e2_df, "E2: Llama-3.1-70B")
e3_metrics, e3_cm, e3_valid = calculate_metrics(e3_df, "E3: FinBERT")

# Create metrics comparison table
metrics_df = pd.DataFrame([e1_metrics, e2_metrics, e3_metrics])

print("\n" + "=" * 80)
print("ZERO-SHOT PERFORMANCE COMPARISON")
print("=" * 80)
display(
    metrics_df[
        ["Experiment", "Accuracy", "Macro-F1", "Macro-Precision", "Macro-Recall"]
    ].round(4)
)

## 6. Visualize Results

In [ ]:
# Performance comparison bar chart
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Overall metrics
metrics_to_plot = ["Accuracy", "Macro-F1", "Macro-Precision", "Macro-Recall"]
x = np.arange(len(metrics_to_plot))
width = 0.25

for i, (metrics, label) in enumerate(
    [
        (e1_metrics, "Mixtral-8x7B"),
        (e2_metrics, "Llama-3.1-70B"),
        (e3_metrics, "FinBERT"),
    ]
):
    values = [metrics[m] for m in metrics_to_plot]
    axes[0].bar(x + i * width, values, width, label=label, alpha=0.8)

axes[0].set_xlabel("Metrics", fontsize=12, weight="bold")
axes[0].set_ylabel("Score", fontsize=12, weight="bold")
axes[0].set_title(
    "Overall Performance Comparison (Zero-Shot)", fontsize=14, weight="bold"
)
axes[0].set_xticks(x + width)
axes[0].set_xticklabels(metrics_to_plot)
axes[0].legend()
axes[0].set_ylim([0, 1])
axes[0].grid(axis="y", alpha=0.3)

# Per-class F1 scores
classes = ["Positive", "Negative", "Neutral"]
x2 = np.arange(len(classes))

for i, (metrics, label) in enumerate(
    [
        (e1_metrics, "Mixtral-8x7B"),
        (e2_metrics, "Llama-3.1-70B"),
        (e3_metrics, "FinBERT"),
    ]
):
    values = [metrics[f"{c}_F1"] for c in classes]
    axes[1].bar(x2 + i * width, values, width, label=label, alpha=0.8)

axes[1].set_xlabel("Sentiment Class", fontsize=12, weight="bold")
axes[1].set_ylabel("F1 Score", fontsize=12, weight="bold")
axes[1].set_title("Per-Class F1 Scores (Zero-Shot)", fontsize=14, weight="bold")
axes[1].set_xticks(x2 + width)
axes[1].set_xticklabels(classes)
axes[1].legend()
axes[1].set_ylim([0, 1])
axes[1].grid(axis="y", alpha=0.3)

plt.tight_layout()
plt.savefig("zero_shot_performance_comparison.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Confusion matrices
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
labels = ["Positive", "Negative", "Neutral"]

for idx, (cm, title) in enumerate(
    [
        (e1_cm, "E1: Mixtral-8x7B"),
        (e2_cm, "E2: Llama-3.1-70B"),
        (e3_cm, "E3: FinBERT"),
    ]
):
    sns.heatmap(
        cm,
        annot=True,
        fmt="d",
        cmap="Blues",
        xticklabels=labels,
        yticklabels=labels,
        ax=axes[idx],
        cbar_kws={"label": "Count"},
    )
    axes[idx].set_title(title, fontsize=12, weight="bold")
    axes[idx].set_ylabel("True Label", fontsize=11, weight="bold")
    axes[idx].set_xlabel("Predicted Label", fontsize=11, weight="bold")

plt.suptitle(
    "Confusion Matrices - Zero-Shot Sentiment Analysis",
    fontsize=14,
    weight="bold",
    y=1.02,
)
plt.tight_layout()
plt.savefig("zero_shot_confusion_matrices.png", dpi=300, bbox_inches="tight")
plt.show()

## 7. Classification Reports

In [ ]:
# Print detailed classification reports
for df_result, exp_name in [
    (e1_valid, "E1: Mixtral-8x7B"),
    (e2_valid, "E2: Llama-3.1-70B"),
    (e3_valid, "E3: FinBERT"),
]:
    print("\n" + "=" * 80)
    print(f"CLASSIFICATION REPORT: {exp_name}")
    print("=" * 80)
    print(
        classification_report(
            df_result["true_sentiment"],
            df_result["predicted_sentiment"],
            labels=["positive", "negative", "neutral"],
            target_names=["Positive", "Negative", "Neutral"],
        )
    )

# Class-wise Metrics Summary Table
print("\n" + "=" * 80)
print("CLASS-WISE METRICS SUMMARY")
print("=" * 80)

summary_data = []
for metrics, model in [
    (e1_metrics, "Mixtral-8x7B"),
    (e2_metrics, "Llama-3.1-70B"),
    (e3_metrics, "FinBERT"),
]:
    summary_data.append(
        {
            "Model": model,
            "Pos_P": metrics["Positive_Precision"],
            "Pos_R": metrics["Positive_Recall"],
            "Pos_F1": metrics["Positive_F1"],
            "Neg_P": metrics["Negative_Precision"],
            "Neg_R": metrics["Negative_Recall"],
            "Neg_F1": metrics["Negative_F1"],
            "Neu_P": metrics["Neutral_Precision"],
            "Neu_R": metrics["Neutral_Recall"],
            "Neu_F1": metrics["Neutral_F1"],
        }
    )

summary_df = pd.DataFrame(summary_data)
print("\nPer-Class Metrics (P=Precision, R=Recall, F1=F1-Score):")
display(summary_df.round(4))

## 8. Save Results

In [ ]:
# Save detailed results
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

e1_df.to_csv(f"e1_mixtral_8x7b_zero_shot_{timestamp}.csv", index=False)
e2_df.to_csv(f"e2_llama_3_1_70b_zero_shot_{timestamp}.csv", index=False)
e3_df.to_csv(f"e3_finbert_zero_shot_{timestamp}.csv", index=False)

# Save metrics summary
metrics_df.to_csv(f"zero_shot_metrics_summary_{timestamp}.csv", index=False)

print(f"\n✓ Results saved with timestamp: {timestamp}")
print("\nFiles created:")
print(f"  - e1_mixtral_8x7b_zero_shot_{timestamp}.csv")
print(f"  - e2_llama_3_1_70b_zero_shot_{timestamp}.csv")
print(f"  - e3_finbert_zero_shot_{timestamp}.csv")
print(f"  - zero_shot_metrics_summary_{timestamp}.csv")
print(f"  - zero_shot_performance_comparison.png")
print(f"  - zero_shot_confusion_matrices.png")

## 9. Error Analysis

In [ ]:
# Error Analysis: Most Common Misclassifications
print("=" * 80)
print("ERROR ANALYSIS: TOP MISCLASSIFIED PATTERNS")
print("=" * 80)

for df_result, exp_name in [
    (e1_valid, "E1: Mixtral-8x7B"),
    (e2_valid, "E2: Llama-3.1-70B"),
    (e3_valid, "E3: FinBERT"),
]:
    print(f"\n{exp_name}")
    print("-" * 80)

    # Identify errors
    errors = df_result[df_result["true_sentiment"] != df_result["predicted_sentiment"]]

    # Count error types
    error_types = (
        errors.groupby(["true_sentiment", "predicted_sentiment"])
        .size()
        .reset_index(name="count")
    )
    error_types = error_types.sort_values("count", ascending=False)

    print(
        f"\nTotal Errors: {len(errors)} / {len(df_result)} ({len(errors) / len(df_result) * 100:.2f}%)"
    )
    print("\nMost Common Error Types:")
    display(error_types.head(5))

    # Show examples of worst errors (high confidence, wrong prediction)
    if len(errors) > 0:
        worst_errors = errors.nlargest(3, "confidence")
        print(f"\nTop 3 High-Confidence Errors:")
        for idx, row in worst_errors.iterrows():
            print(
                f"\n  True: {row['true_sentiment']} | Predicted: {row['predicted_sentiment']} | Conf: {row['confidence']:.2f}"
            )
            print(f"  Sentence: {row['sentence'][:120]}...")
            print(f"  Rationale: {row['rationale']}")

# Class-wise Performance Comparison
print("=" * 80)
print("CLASS-WISE PERFORMANCE BREAKDOWN")
print("=" * 80)

class_comparison = []
for metrics, model_name in [
    (e1_metrics, "Mixtral-8x7B"),
    (e2_metrics, "Llama-3.1-70B"),
    (e3_metrics, "FinBERT"),
]:
    for sentiment in ["Positive", "Negative", "Neutral"]:
        class_comparison.append(
            {
                "Model": model_name,
                "Class": sentiment,
                "Precision": metrics[f"{sentiment}_Precision"],
                "Recall": metrics[f"{sentiment}_Recall"],
                "F1-Score": metrics[f"{sentiment}_F1"],
            }
        )

class_df = pd.DataFrame(class_comparison)

# Pivot for better visualization
for metric in ["Precision", "Recall", "F1-Score"]:
    print(f"\n{metric} by Class:")
    pivot = class_df.pivot(index="Class", columns="Model", values=metric)
    display(pivot.round(4))

# Identify weakest class per model
print("\n" + "=" * 80)
print("WEAKEST CLASS IDENTIFICATION")
print("=" * 80)
for metrics, model_name in [
    (e1_metrics, "Mixtral-8x7B"),
    (e2_metrics, "Llama-3.1-70B"),
    (e3_metrics, "FinBERT"),
]:
    f1_scores = {
        "Positive": metrics["Positive_F1"],
        "Negative": metrics["Negative_F1"],
        "Neutral": metrics["Neutral_F1"],
    }
    weakest = min(f1_scores, key=f1_scores.get)
    print(f"{model_name}: Weakest class = {weakest} (F1={f1_scores[weakest]:.4f})")

## 10. Confidence Analysis

In [ ]:
# Confidence analysis
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (df_result, title) in enumerate(
    [(e1_valid, "Mixtral-8x7B"), (e2_valid, "Llama-3.1-70B"), (e3_valid, "FinBERT")]
):
    df_result["correct"] = (
        df_result["true_sentiment"] == df_result["predicted_sentiment"]
    )

    correct_conf = df_result[df_result["correct"]]["confidence"]
    incorrect_conf = df_result[~df_result["correct"]]["confidence"]

    axes[idx].hist(
        [correct_conf, incorrect_conf],
        bins=20,
        label=["Correct", "Incorrect"],
        alpha=0.7,
        color=["green", "red"],
    )
    axes[idx].set_xlabel("Confidence Score", fontsize=11, weight="bold")
    axes[idx].set_ylabel("Frequency", fontsize=11, weight="bold")
    axes[idx].set_title(
        f"{title}\nMean Conf: Correct={correct_conf.mean():.3f}, Incorrect={incorrect_conf.mean():.3f}",
        fontsize=11,
        weight="bold",
    )
    axes[idx].legend()
    axes[idx].grid(alpha=0.3)

plt.suptitle(
    "Confidence Distribution: Correct vs Incorrect Predictions",
    fontsize=14,
    weight="bold",
    y=1.02,
)
plt.tight_layout()
plt.savefig("zero_shot_confidence_analysis.png", dpi=300, bbox_inches="tight")
plt.show()

print("\n" + "=" * 80)
print("CONFIDENCE CALIBRATION ANALYSIS")
print("=" * 80)
for df_result, exp_name in [
    (e1_valid, "E1: Mixtral-8x7B"),
    (e2_valid, "E2: Llama-3.1-70B"),
    (e3_valid, "E3: FinBERT"),
]:
    df_result["correct"] = (
        df_result["true_sentiment"] == df_result["predicted_sentiment"]
    )

    avg_conf_correct = df_result[df_result["correct"]]["confidence"].mean()
    avg_conf_incorrect = df_result[~df_result["correct"]]["confidence"].mean()
    calibration_gap = avg_conf_correct - avg_conf_incorrect

    print(f"\n{exp_name}:")
    print(f"  Average Confidence (Correct): {avg_conf_correct:.4f}")
    print(f"  Average Confidence (Incorrect): {avg_conf_incorrect:.4f}")
    print(f"  Calibration Gap: {calibration_gap:.4f}")
    print(f"  Total Correct: {df_result['correct'].sum()} / {len(df_result)}")

    # Confidence by sentiment class
    print(f"\n  Confidence by Predicted Class:")
    for sentiment in ["positive", "negative", "neutral"]:
        class_df = df_result[df_result["predicted_sentiment"] == sentiment]
        if len(class_df) > 0:
            print(
                f"    {sentiment.capitalize()}: {class_df['confidence'].mean():.4f} (n={len(class_df)})"
            )

## 11. Key Findings & Conclusions

### Expected Conclusions from Zero-Shot Experiment

### 1. **Model Performance Comparison**
   - **Best Performer**: Identify which model (Mixtral-8x7B, 120B, or FinBERT) achieves highest Macro-F1
   - **Accuracy vs F1**: Check if high accuracy masks poor performance on minority class (negative: 13%)
   - **Consistency**: Compare Macro-F1 vs Weighted-F1 to assess class imbalance handling

### 2. **Class-Specific Insights**
   - **Negative Sentiment Challenge**: Expect lowest F1 on negative class (only 297/2,217 examples = 13.4%)
   - **Positive vs Neutral Confusion**: Analyze if models confuse growth statements with neutral facts
   - **Per-Class Ranking**: Typically: Positive > Neutral > Negative (due to data distribution)

### 3. **Zero-Shot Limitations**
   - **No Examples Handicap**: Models rely purely on pretrained financial knowledge
   - **Expected Baseline**: Should be lower than Few-Shot, CoT, and ToT approaches
   - **Benchmark Value**: Establishes minimum performance without prompt engineering

### 4. **Prompt Engineering Impact**
   - **Improved vs Original**: Compare new prompt results against previous 100-sample run
   - **Negative Detection**: Check if explicit negative indicators (losses, declines) improved recall
   - **JSON Compliance**: Monitor parsing error rate

### 5. **Confidence Calibration Analysis**
   - **Overconfidence**: Models often show high confidence on incorrect predictions
   - **Class-Specific Confidence**: Positive predictions typically more confident than negative
   - **Calibration Gap**: Measure difference between confidence and actual accuracy

### 6. **Cross-Model Patterns**
   - **Size Matters**: Does Llama-3.1-70B outperform 20B significantly?
   - **Architecture Differences**: Llama vs GPT-style models on financial domain
   - **Error Consistency**: Do all models make similar mistakes?

### 7. **Actionable Recommendations**
   - If Negative F1 < 0.3: Zero-shot insufficient, must use Few-Shot with negative examples
   - If Macro-F1 < 0.6: Implement CoT to force reasoning about financial indicators
   - If high confidence on errors: Add uncertainty quantification to prompts

### 8. **Business Impact Assessment**
   - **Deployment Readiness**: Macro-F1 > 0.75 needed for production use
   - **Cost-Benefit**: Compare API costs vs performance gains from advanced prompting
   - **Risk Analysis**: False negatives (missing bad news) more costly than false positives